# DEMO: Pipeline RAG PDF — End-to-End

Ce notebook met en œuvre un pipeline complet de type RAG (Retrieval-Augmented Generation), spécifiquement conçu pour interroger, comprendre et exploiter automatiquement des documents contractuels au format PDF de la MAIF

## 1. Configuration de l'environnement


In [ ]:
import os
import sys

# Ajouter le dossier racine du projet au path Python
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

## 2. Imports des modules du pipeline

In [ ]:
from core.config import PDF_DIR, INDEX_PATH, METADATA_PATH
from core.loader import load_pdfs
from core.chunker import chunk_texts  
from core.embedder import embed_chunks
from core.vector_store import FaissIndex
from core.retriever import RAGRetriever
from core.generator import generate_answer



## 3. Chargement des documents PDF

- Lecture de tous les fichiers PDF dans le répertoire `PDF_DIR`.



In [10]:

docs = load_pdfs(PDF_DIR)
print(f"✅ {len(docs)} documents PDF chargés.")

✅ 5 documents PDF chargés.


## 4. Chunking des textes



In [11]:
chunks = chunk_texts(docs)
print(f"✅ {len(chunks)} chunks générés.")

✅ 1845 chunks générés.


## 5. Calcul des embeddings

- Transformation des chunks en vecteurs via le modèle d'embeddings.

In [12]:
embeddings = embed_chunks(chunks)
# Vérification de la forme
import numpy as np
embeddings = np.asarray(embeddings, dtype='float32')
print(f"✅ {embeddings.shape[0]} vecteurs créés (dimension = {embeddings.shape[1]}).")

✅ 1845 vecteurs créés (dimension = 384).


## 6. Construction de l'index FAISS

- Création d'un index et ajout des embeddings et métadonnées.

In [13]:
index = FaissIndex(dim=embeddings.shape[1])
index.add(embeddings, metadatas=chunks)
print(f"✅ Index FAISS créé avec {index.index.ntotal} vecteurs.")

✅ Index FAISS créé avec 1845 vecteurs.


## 7. Sauvegarde de l'index et des métadonnées



In [14]:
os.makedirs(os.path.dirname(INDEX_PATH), exist_ok=True)
index.save(INDEX_PATH, METADATA_PATH)
print("✅ Index et métadonnées sauvegardés avec succès.")

✅ Index et métadonnées sauvegardés avec succès.


## 8. Exemple de requête RAG



In [ ]:
# Initialisation du retriever
retriever = RAGRetriever()

# Query
query = "Comment sont annexés les plans locaux d'urbanisme ?"

# Récupération des chunks
results = retriever.retrieve(query, top_k=5)

# Affichage des résultats
print("\n✅ Chunks récupérés :\n")
for i, (chunk, score) in enumerate(results, start=1):
    print(f"Chunk {i} — Doc : {chunk.get('doc_id')} — Score : {score:.4f}")
    print(chunk.get('text')[:300].replace('\n', ' '))
    print("—" * 80)


✅ Chunks récupérés :

Chunk 1 — Doc : Conditions+generales+Maif+assurance+habitation+Raqvam.pdf — Score : 0.6390
en œuvre les moyens de prévention et de protection définis par les lois et règlements en vigueur, en particulier par les plans de prévention des risques naturels. Ces plans sont annexés aux plans  locaux d’urbanisme (anciennement plans d’occupation des sols) et font l’objet d’un affichage en  mairie
————————————————————————————————————————————————————————————————————————————————
Chunk 2 — Doc : aha-cg-filia.pdf — Score : 0.7843
prévention des risques naturels. Ces plans sont annexés aux plans locaux d’urbanisme (anciennement plans  d’occupation des sols) et font l’objet d’un affichage en mairie et d’une publicité par voie de presse locale. En cas de manquement de votre part à ces obligations, MAIF est fondée à vous réclame
————————————————————————————————————————————————————————————————————————————————
Chunk 3 — Doc : conditions-generales-assurance-protection-juridique-maif

## 9. Génération de réponse via LLM

- Utilisation des chunks récupérés pour produire une réponse contextualisée.

In [19]:

answer = generate_answer(results, query)
print("\n Réponse générée :\n")
print(answer)



 Réponse générée :

Les plans locaux d'urbanisme sont annexés aux plans de prévention des risques naturels, eux-mêmes annexés aux plans locaux d'urbanisme qui étaient auparavant appelés plans d'occupation des sols (passages 1, 2 et 5). Ces annexions sont rendues publiques par un affichage en mairie et une publicité dans la presse locale.
